In [1]:
import pandas

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load datasets
dataset_train = pd.read_csv("./data/train.csv").drop(columns="label")
label_train = pd.read_csv("./data/train.csv")["label"]
dataset_test = pd.read_csv("./data/test.csv")